In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191022174824-0000
KERNEL_ID = 550115ca-6dad-4079-a9ff-bebaaeff8551


# <center>Compute Telemtry Facts</center>

In this notebook we will see how to compute the trip facts from telemetry points stored in SESA's bucket for raw telemetry. We will load telemetry from October 2018 to May 2019 and perform mathematical operations to summarise a trip.

First, let's load the needed libraries.

In [2]:
# Spark required imports
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import Window
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import when

# Import seaborn and matplotlib for data exploration/visialization.
#!pip install --upgrade seaborn

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#import pixiedust

# Access to IBM Cloud Object Storage
import ibmos2spark
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

# Python Utilities
from datetime import datetime

# JSON
import json

# pandas options 
pd.options.display.max_rows = 999
pd.set_option('float_format', '{:f}'.format)

## Service Credentials

In [3]:
# The code was removed by Watson Studio for sharing.

## Load telemetry by month

In [4]:
Trips = spark.read.parquet(local_cos.url("TripFacts_V5_Omar_clean", local_bucket))
print(Trips.count())

3202976


In [5]:
Trips = Trips[Trips.DeviceId.isin([32347, 32248, 36605, 32142, 36080, 36218, 35690, 29931, 41016,37458, 40013, 34055, 30706, 31677, 30919, 32551, 32508, 34485,
       31536, 36584, 29899, 36761, 37211, 29635, 40948, 34828, 38988,31202, 40568, 38340, 29903, 30579, 31195, 30770, 41487, 41887,
       43377, 41591, 32266, 30259, 42116, 35797, 35819, 35039, 34285, 33493, 41414, 43278, 42813, 36143, 30487, 27940, 40197, 31421,
       29369, 39383, 29619, 33347, 36216, 38635, 31664]) == False]

Trips.count()

3174941

In [6]:
#Load GEOJSON

Ecuador_geo = spark.read.json(local_cos.url("ecu_admbda_adm3_inec_20190724.json.geojson", local_bucket))
Ecuador_geo= Ecuador_geo.select("properties.ADM3_ES","properties.ADM3_PCODE","properties.ADM2_ES","properties.ADM2_PCODE","properties.ADM1_ES","properties.ADM1_PCODE","properties.ADM0_ES")
Ecuador_geo= Ecuador_geo.where(F.col("ADM3_PCODE").isNotNull())
Ecuador_geo.where(F.col("ADM1_ES")=='Pichincha').show(truncate = False)

+----------------------+----------+----------+----------+---------+----------+------------+
|ADM3_ES               |ADM3_PCODE|ADM2_ES   |ADM2_PCODE|ADM1_ES  |ADM1_PCODE|ADM0_ES     |
+----------------------+----------+----------+----------+---------+----------+------------+
|Alangasi              |EC170151  |Quito     |EC1701    |Pichincha|EC17      |Ecuador (el)|
|Aloag                 |EC170351  |Mejia     |EC1703    |Pichincha|EC17      |Ecuador (el)|
|Aloasi                |EC170352  |Mejia     |EC1703    |Pichincha|EC17      |Ecuador (el)|
|AmaguaÃ±a             |EC170152  |Quito     |EC1701    |Pichincha|EC17      |Ecuador (el)|
|Ascazubi              |EC170251  |Cayambe   |EC1702    |Pichincha|EC17      |Ecuador (el)|
|Atahualpa (Habaspamba)|EC170153  |Quito     |EC1701    |Pichincha|EC17      |Ecuador (el)|
|Calacali              |EC170154  |Quito     |EC1701    |Pichincha|EC17      |Ecuador (el)|
|Calderon (Carapungo)  |EC170155  |Quito     |EC1701    |Pichincha|EC17      |Ec

### Read trips with geographic information 

In [ ]:
#Geografia_Enero = spark.read\
#  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#  .option('header', 'true')\
#  .option('inferSchema', 'true')\
#  .load(local_cos.url('ResultGeoData_1.csv', local_bucket))
#print(Geografia_Enero.count())

#Geografia_Febrero = spark.read\
#  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#  .option('header', 'true')\
#  .option('inferSchema', 'true')\
#  .load(local_cos.url('ResultGeoData_2.csv', local_bucket))
#print(Geografia_Febrero.count())

#Geografia_Marzo = spark.read\
#  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#  .option('header', 'true')\
#  .option('inferSchema', 'true')\
#  .load(local_cos.url('ResultGeoData_3.csv', local_bucket))
#print(Geografia_Marzo.count())

#Geografia_Abril = spark.read\
#  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#  .option('header', 'true')\
#  .option('inferSchema', 'true')\
#  .load(local_cos.url('ResultGeoData_4.csv', local_bucket))
#print(Geografia_Abril.count())


#Geografia_Mayo = spark.read\
#  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#  .option('header', 'true')\
#  .option('inferSchema', 'true')\
#  .load(local_cos.url('ResultGeoData_5.csv', local_bucket))
#print(Geografia_Mayo.count())


#Geografia_Junio = spark.read\
#  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#  .option('header', 'true')\
#  .option('inferSchema', 'true')\
#  .load(local_cos.url('ResultGeoData_6.csv', local_bucket))
#print(Geografia_Junio.count())


#Geografia_Julio = spark.read\
#  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#  .option('header', 'true')\
#  .option('inferSchema', 'true')\
#  .load(local_cos.url('ResultGeoData_7.csv', local_bucket))
#print(Geografia_Julio.count())


#Geografia_Agosto = spark.read\
#  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#  .option('header', 'true')\
#  .option('inferSchema', 'true')\
#  .load(local_cos.url('ResultGeoData_8.csv', local_bucket))
#print(Geografia_Agosto.count())

#Geografia_Septiembre = spark.read\
#  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
#  .option('header', 'true')\
#  .option('inferSchema', 'true')\
#  .load(local_cos.url('ResultGeoData_9.csv', local_bucket))
#print(Geografia_Septiembre.count())

Geografia = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(local_cos.url('ResultGeoData_full.csv',local_bucket))
Geografia.count()

In [8]:
Geografia.show()

+--------+--------+------------+-----------+-------------+------------+--------+--------+
|  TripID|DeviceId|Longitude_ON|Latitude_ON|Longitude_OFF|Latitude_OFF|   ON_ID|  OFF_ID|
+--------+--------+------------+-----------+-------------+------------+--------+--------+
|27866888|   39597|   -78.46661|   -0.06708|  -78.4987946|   -0.134649|EC170177|EC170150|
|27866890|   39965| -79.4852753|  -3.314891|    -79.71637|   -3.327999|EC010650|EC070952|
|27866892|   38115|   -80.39758|  -2.635217|    -80.39264|   -2.636378|EC092150|EC092150|
|27866900|   30931|   -79.92662|  -2.199337|    -79.93083|   -2.198119|EC090150|EC090150|
|27866904|   35404|   -78.55334|  -0.334128|  -78.4696045|   -0.191106|EC170150|EC170150|
|27866907|   37775| -80.0097656|   -2.18636|    -79.89834|   -2.170095|EC090150|EC090150|
|27866919|   39351|   -79.05156|  -2.920258|    -78.95212|   -2.884877|EC010150|EC010150|
|27866928|   35639|   -78.44177|  -0.009574|    -78.49089|   -0.106988|EC170180|EC170150|
|27866930|

In [9]:
#Geografia = Geografia_Enero.union(Geografia_Febrero).union(Geografia_Marzo).union(Geografia_Abril).union(Geografia_Mayo).union(Geografia_Junio).union(Geografia_Julio).union(Geografia_Agosto)#.union(Geografia_Septiembre)
Geografia = Geografia.select('DeviceId', 'TripID', 'ON_ID', 'OFF_ID')
Geografia = Geografia.withColumn('ON_ID', F.regexp_replace(F.col('ON_ID'),"[\[\]']" , ''))
Geografia = Geografia.withColumn('OFF_ID', F.regexp_replace(F.col('OFF_ID'),"[\[\]']" , ''))
#Geografia.show()

Join geometry codes with trip facts data set.

In [10]:
# join
Trips_Geog = Trips.join(Geografia, ['DeviceId', 'TripID'], 'inner')
Trips_Geog = Trips_Geog.join(Ecuador_geo.selectExpr("ADM3_ES as ON_N3","ADM2_ES as ON_N2", "ADM1_ES as ON_N1", 'ADM3_PCODE'), Trips_Geog['ON_ID'] == Ecuador_geo["ADM3_PCODE"], 'left_outer').drop("ADM3_PCODE")
Trips_Geog = Trips_Geog.join(Ecuador_geo.selectExpr("ADM3_ES as OFF_N3","ADM2_ES as OFF_N2", "ADM1_ES as OFF_N1", 'ADM3_PCODE'), Trips_Geog['OFF_ID'] == Ecuador_geo["ADM3_PCODE"], 'left_outer').drop("ADM3_PCODE")
Trips_Geog.show()

+--------+--------+------------------+-------------------+------------------+-------------------+-------------------+--------+--------+--------------------+--------------------+------------------+-------+-------+------------------+-------------------+-----------+------------+-------------------+------------+-------------+-------------------+-----------+-------+-------+-----+----------+--------+--------+------------+-------+--------+------------+-------+--------+
|DeviceId|  TripID|           Dist_km|         Duration_h|       DistanciaST|       DuracionST_h|           Dur_50_h|Dur_70_h|Dur_80_h|            Dur_90_h|           Dur_100_h|           Dist_50|Dist_70|Dist_80|           Dist_90|           Dist_100|Latitude_ON|Longitude_ON|         EC_Time_ON|Latitude_OFF|Longitude_OFF|        EC_Time_OFF|    DayTime|DayType|DayWeek|Month|      Date|   ON_ID|  OFF_ID|       ON_N3|  ON_N2|   ON_N1|      OFF_N3| OFF_N2|  OFF_N1|
+--------+--------+------------------+-------------------+--------

In [11]:
Trips_Geog = Trips_Geog.withColumn('ViajeInterparroquial', F.when(F.col('ON_N3')==F.col('OFF_N3'), 0).otherwise(1))
Trips_Geog = Trips_Geog.withColumn('ViajeIntercantonal', F.when(F.col('ON_N2')==F.col('OFF_N2'), 0).otherwise(1))
Trips_Geog = Trips_Geog.withColumn('ViajeInterprovincial', F.when(F.col('ON_N1')==F.col('OFF_N1'), 0).otherwise(1))
Trips_Geog.show()

+--------+--------+------------------+-------------------+------------------+-------------------+--------------------+--------+-------------------+--------------------+-------------------+------------------+-------+------------------+------------------+------------------+-----------+------------+-------------------+------------+-------------+-------------------+-----------+-------+-------+-----+----------+--------+--------+--------------------+--------+--------+------------+-------+--------+--------------------+------------------+--------------------+
|DeviceId|  TripID|           Dist_km|         Duration_h|       DistanciaST|       DuracionST_h|            Dur_50_h|Dur_70_h|           Dur_80_h|            Dur_90_h|          Dur_100_h|           Dist_50|Dist_70|           Dist_80|           Dist_90|          Dist_100|Latitude_ON|Longitude_ON|         EC_Time_ON|Latitude_OFF|Longitude_OFF|        EC_Time_OFF|    DayTime|DayType|DayWeek|Month|      Date|   ON_ID|  OFF_ID|               

In [12]:
Trips_Geog.count()

3174941

In [6]:
# Save data
Trips_Geog.write.parquet(local_cos.url("TripFacts5_Geo", local_bucket), mode = 'overwrite')
project.save_data('Datos_Geo.csv', Trips_Geog.toPandas().to_csv(header=True, index=False), overwrite = True) 

In [4]:
Trips_Geog = spark.read.parquet(local_cos.url("TripFacts5_Geo", local_bucket))

In [5]:
Trips_Geog.printSchema()

root
 |-- DeviceId: long (nullable = true)
 |-- TripID: long (nullable = true)
 |-- Dist_km: double (nullable = true)
 |-- Duration_h: double (nullable = true)
 |-- DistanciaST: double (nullable = true)
 |-- DuracionST_h: double (nullable = true)
 |-- Dur_50_h: double (nullable = true)
 |-- Dur_70_h: double (nullable = true)
 |-- Dur_80_h: double (nullable = true)
 |-- Dur_90_h: double (nullable = true)
 |-- Dur_100_h: double (nullable = true)
 |-- Dist_50: double (nullable = true)
 |-- Dist_70: double (nullable = true)
 |-- Dist_80: double (nullable = true)
 |-- Dist_90: double (nullable = true)
 |-- Dist_100: double (nullable = true)
 |-- Latitude_ON: double (nullable = true)
 |-- Longitude_ON: double (nullable = true)
 |-- EC_Time_ON: timestamp (nullable = true)
 |-- Latitude_OFF: double (nullable = true)
 |-- Longitude_OFF: double (nullable = true)
 |-- EC_Time_OFF: timestamp (nullable = true)
 |-- DayTime: string (nullable = true)
 |-- DayType: string (nullable = true)
 |-- DayWee